In [2]:
import pandas as pd
import math
from itertools import product
import datetime

In [164]:
## provide inputs below
temp = {"setpt_occ":72, "setpt_unocc":65, "SAT":55} # un/occupied room setpoints & supply air temp (F)
ACH = {"min_unocc":4, "aircuity":4}  # minimum unoccupied & aircuity ACH

location = 'Madison'

design_heating = {"DB":-6.8} # design heating conditions (F)
design_cooling = {"DB":89.2, "W":0.01463} # design cooling conditions (F, lb/lb)

DA = {"DB": 50, "W":0.00624} # discharge air setpoint (F, lb/lb)
EA = {"DB":74, "W":0.00896, "h":27.566, "v":13.643} # exhaust air conditions (F, lb/lb, btu/lb, cf/lb)

effectiveness_runaround = {"sensible":0.4,"latent":0} # effectiveness for runaround energy recovery
effectiveness_ERW = {"sensible":0.65,"latent":0.6} # effectiveness for energy recovery wheel

# cfm_transfer = 150 # if airflow offset is constant 

# fume hood characteristics (for a single hood)
# VAV hood here operates at the maximum cfm only when in "set up" position; uses a lower cfm when in "in use" position
fumehood_vals_standard = {"cfm_max":790, "cfm_min":395, "velocity":100}
fumehood_vals_high_performance = {"cfm_max":630, "cfm_min":325, "velocity":80}
point_exhaust_vals = {'cfm':300}

cooling_vals = {"efficiency":0.576, "tower/pump_factor":0.2} # design cooling efficiency (kw/ton), tower/pump energy factor
steam_vals = {"efficiency":0.7, "btu/lb":1133} # steam system efficiency & BTU/LB conversion
heating_vals = {"efficiency":0.8} # heating system efficiency
fan_vals = {"kw/pd/cfm":0.00022} # fan design power (kW/in. wg.- CFM) & pressure drop (in. wg.)

height = 10 # ceiling height (ft)
area = 2731 # floor area (sf)

utility = {'$/kw-peak':0.04145, '$/kw':0.02935, '$/cf':0.00952, '$/MMBTU':8.5455} # utility rates
peak_hours = [9, 15] # start & end for peak hours (assuming weekdays only)
holidays = [datetime.date(2017,1,1),datetime.date(2017,1,16),datetime.date(2017,5,29), # dates for all holidays as (Y, M, D); the year is a placeholder
    datetime.date(2017,7,4),datetime.date(2017,9,4),datetime.date(2017,11,11), datetime.date(2017,11,23),
    datetime.date(2017,11,24),datetime.date(2017,12,25),datetime.date(2017,12,26)]

gas_emissions = {'CO2':53.06, 'CH4':0.001, 'N2O':0.0001} # natural gas emissions factors (kg-gas per mmBtu)
elec_emissions = {'CO2':1526.4, 'CH4':0.139, 'N2O':0.02} # electricity emissions factors (lb/MWh)
GWP = {'CO2':1, 'CH4':25,'N2O':298} # global warming potential for conversion to CO2e

In [165]:
## provide path to excel containing each profile on a separate sheet, named as below
# the tabs should only have the data columns, the dataframe import provides its own index (refer to excel file)
path = 'C:\\Users\\uirfan\\OneDrive - SmithGroup Companies Inc\\Desktop\\14276 - UWM eng\\Reference.xlsx'
profile_OA = pd.read_excel(path, sheet_name=f'Outside Air_{location}')
profile_FH_Airflow_Standard = pd.read_excel(path, sheet_name='FH % Airflow_Standard')
profile_FH_Airflow_HP = pd.read_excel(path, sheet_name='FH % Airflow_HP')
profile_Cooling_PLR = pd.read_excel(path, sheet_name='Cooling Load PLR')
profile_Occupancy = pd.read_excel(path, sheet_name='Occupancy')
profile_PE_Control = pd.read_excel(path, sheet_name='Point Exhaust Control')
PD_vals = pd.read_excel(path, sheet_name='Pressure Drop', index_col=0) # provided for 500 fpm


In [5]:
## provide parameters to be iterated below

FH_R = ['(02) 6 ft', '(04) 6 ft', '(06) 6 ft', '(08) 6 ft', '(12) 6 ft']
FH_Use_Occupancy_R = ['Research - Light', 'Research - Moderate', 'Research - Heavy']
Cooling_Load_WSF_R = [3.81, 5.81, 7.81]
PE_R = [8, 12]
ACH_Min_Occupied_R = [6, 8, 10]

# FH_I = ['(02) 6 ft', '(04) 6 ft', '(08) 6 ft', '(12) 6 ft']
# FH_Use_Occupancy_I = ['Instructional - Light', 'Instructional - Moderate', 'Instructional - Heavy']
# Cooling_Load_WSF_I = [4.07, 6.07, 8.07]
# PE_I = [0, 6]
# ACH_Min_Occupied_I = [6, 8, 10]

FH_I = ['(00) 6 ft']
FH_Use_Occupancy_I = ['Instructional - Light', 'Instructional - Moderate', 'Instructional - Heavy']
Cooling_Load_WSF_I = [4.07, 6.07, 8.07]
PE_I = [0, 6]
ACH_Min_Occupied_I = [4]

Energy_Recovery = ['None', 'Run around', 'Run around + ERW']
Aircuity = [True, False]
Chilled_Beams = [True, False]
AHU_Velocity = [400, 300, 200]
FH_Type = ['Standard', 'High Performance']
PE_Control = ['4 snorkels per switch-controlled group', 'Individual snorkel control']

# create list of all possible parameter cases (cartesian product)
combinations_R = list(product(FH_R, FH_Use_Occupancy_R, FH_Type, Cooling_Load_WSF_R, ACH_Min_Occupied_R, Energy_Recovery, Aircuity, Chilled_Beams, AHU_Velocity, PE_R, PE_Control))
combinations_I = list(product(FH_I, FH_Use_Occupancy_I, FH_Type, Cooling_Load_WSF_I, ACH_Min_Occupied_I, Energy_Recovery, Aircuity, Chilled_Beams, AHU_Velocity, PE_I, PE_Control))

# qty_cases = len(combinations_R) + len(combinations_I)
qty_cases = len(combinations_R)
print(qty_cases) # the number of cases run
print(str(round(qty_cases*0.0585/60,2)) + ' hrs')

38880
37.91 hrs


In [167]:
## create profile for electricity rates
year = 2017
hour_series_pd = pd.Series(pd.date_range(start=f'{year}-01-01', end=f'{year}-12-31 23:00:00', freq='H'))
hour_series = [x.to_pydatetime() for x in list(hour_series_pd)]
elec_utility = [0]

for hr in range(1,8761):
    u = utility['$/kw']
    if hour_series[hr-1].weekday() < 5:
        if hour_series[hr-1].date() not in holidays:
            if (hour_series[hr-1].hour >= peak_hours[0]) and (hour_series[hr-1].hour <= peak_hours[1]):
                u = utility['$/kw-peak']
    elec_utility.append(u)

df_elec_utility = pd.DataFrame(elec_utility)
# print(df_elec_utility)

In [168]:
## main parameter & hourly loops
counter = 0
for q in [combinations_I]:
    cases = [[] for i in range(34)] # create empty lists for output

    for comb in q: 
        counter += 1
        # set totals to 0 at the start of each case loop
        driving_factor = {"Cooling":0, "ACHR":0, "Exhaust":0}
        totals_load = {'cfm_design':0, 'cfm_actual':0,'cooling_tons':0}
        totals_cost = {'cooling':0, 'heating':0, 'humid':0, 'reheat':0, 'fans':0, 'pumps':0}
        totals_energy_cost = {'electricity':0, 'gas':0, 'total':0}
        totals_energy_elec = {'cooling':0, 'fans':0, 'pumps':0, 'sum':0}
        totals_energy_gas = {'heating':0, 'humid':0, 'reheat':0, 'sum': 0}
        total_CO2e = {'cooling':0, 'heating':0, 'humid':0, 'reheat':0, 'fans':0, 'pumps':0, 'sum':0}
        
        if comb[2] == 'Standard':
            fumehood = fumehood_vals_standard
            profile_FH_Airflow = profile_FH_Airflow_Standard
        else: 
            fumehood = fumehood_vals_high_performance
            profile_FH_Airflow = profile_FH_Airflow_HP
        point_exhaust = point_exhaust_vals

        # read parameters from combination list
        fumehood['qty'] = int(comb[0][1:3])
        occupants = comb[1]
        cooling_vals['load_wsf'] = comb[3]
        ACH["min_occ"] = comb[4]
        recovery = comb[5]
        aircuity = comb[6] 
        CHB = comb[7]
        ahu_v = comb[8]
        point_exhaust['qty'] = comb[9]
        point_exhaust['profile'] = comb[10]
        
        # add case parameters to output list
        cases[0].append(comb[0])
        cases[1].append(occupants[:occupants.find('-')-1])
        cases[2].append(occupants[occupants.find('-')+2:])
        for i in range(2,11):
            cases[i+1].append(comb[i])

        # calculate component pressure drops based on new fpm
        velocity_factor = (ahu_v/500)**2 # PD1/PD2 = (CFM1/CFM2)^2
        PD_vals_current = PD_vals*velocity_factor
        if recovery == "None":
            PD_vals_current.drop(['Energy Recovery Coil','Energy Recovery Wheel'], inplace = True)         
        elif recovery == "Run around":
            PD_vals_current.drop(['Energy Recovery Wheel'], inplace = True)
        else: pass
        
        # airflow offset is dependent on the number of fume hoods here
        if fumehood['qty'] < 3:  
            cfm_transfer = 150
        else:
            cfm_transfer = 250

        for hour in range(1,8761): # hourly loop

            # read the profile values at the current hour and occupancy type
            cooling_vals["PLR"] = profile_Cooling_PLR.at[hour, occupants[:occupants.find('-')-1]]
            fumehood['cfm_fraction'] = profile_FH_Airflow.at[hour, occupants]
            OA = {"DB":profile_OA.at[hour, "DB"], "W":profile_OA.at[hour, "W"], "h":profile_OA.at[hour, "h"], "v":profile_OA.at[hour, "v"]}
            occupancy = profile_Occupancy.at[hour, occupants[:occupants.find('-')-1]]
            point_exhaust['fraction'] = profile_PE_Control.at[hour, point_exhaust['profile']]

            # calculate airflow based on cooling, fume hood exhaust, and air changes requirement
            cfm_current = {'cooling':airflow_cooling_hourly(cooling_vals, area, temp, occupancy),
                          'FHE':airflow_LE_hourly(fumehood, point_exhaust, cfm_transfer),
                          'ACHR':airflow_ACHR_hourly(ACH, height, area, aircuity, occupancy)}

            # set driving factor using maximum airflow, excluding cooling if chilled beams are present
            if CHB:
                if cfm_current['FHE'] > cfm_current['ACHR']:
                    driver = 'FHE'
                else: 
                    driver = 'ACHR'
            else:
                driver = max(cfm_current, key = cfm_current.get)
            cfm_current["final"] = cfm_current[driver] # add entry for current airflow based on driving factor
            cfm_current["GE"] = max(0, cfm_current["final"]-airflow_LE_hourly(fumehood, point_exhaust, 0)) # general exhaust cfm for energy recovery wheel

            # keep count of hours for each driving factor
            if driver == "cooling":
                driving_factor["Cooling"] += 1
            elif driver == "ACHR":
                driving_factor["ACHR"] += 1
            else:
                driving_factor["Exhaust"] += 1 

            # same process as above for design airflow (except not accounting for chilled beams)
            cfm_design = {'cooling':airflow_cooling_design(cooling_vals["load_wsf"], area, temp),
                      'FHE':airflow_LE_design(fumehood, point_exhaust, cfm_transfer),
                      'ACHR':airflow_ACHR_design(ACH, height, area)}
            cfm_design["final"] = cfm_design[max(cfm_design, key = cfm_design.get)]
            cfm_design["GE"] = max(0, cfm_design["final"] + cfm_transfer - fumehood["cfm_min"]*fumehood['qty']) # general exhaust cfm for energy recovery wheel

            # calculate current part load ratio
            airflow_PLR = cfm_current["final"]/cfm_design["final"]
            LE_airflow_PLR = airflow_PLR # in case of no ERW, all exhaust goes to LEF
            GE_airflow_PLR = 0 # in case of no ERW
            
            # run energy recovery calculation for the three scenarios
            if recovery == "None":
                coil_EA = OA         
            elif recovery == "Run around":
                if driver == 'FHE':
                    x = energy_recovery(effectiveness_runaround, OA, cfm_current["final"], EA, airflow_LE_hourly(fumehood, point_exhaust, 0))
                    coil_EA = adjust_recovery(x, OA, DA)
                else:
                    x = energy_recovery(effectiveness_runaround, OA, cfm_current["final"], EA, cfm_current["final"])
                    coil_EA = adjust_recovery(x, OA, DA)
            else:
                # assuming runaround upstream of wheel, runaround leaving air conditions = wheel entering air conditions
                # fume hood exhaust through runaround & general exhaust through wheel
                x = energy_recovery(effectiveness_runaround, OA, cfm_current["final"], EA, airflow_LE_hourly(fumehood, point_exhaust, 0))
                y = energy_recovery(effectiveness_ERW, x, cfm_current["final"], EA, cfm_current["GE"])
                coil_EA = adjust_recovery(y, OA, DA)
                LE_airflow_PLR = cfm_current['FHE']/cfm_design["FHE"] # if ERW is present, FHE goes to LEF, GE goes to GEF
                GE_airflow_PLR = cfm_current['GE']/cfm_design["GE"]
                
            # calculate current cooling load (chilled beams, sensible, latent)
            cooling_load = load_cooling(cfm_current, coil_EA, DA) 
            cooling_load["CHB"] = max(0, 1.08*(cfm_current['cooling']-cfm_current['final'])*(temp["setpt_occ"]-temp["SAT"])) # calculate chilled beams cooling load
            cooling_load["total"] = sum(cooling_load.values())/12000 # sum the 3 loads and convert to tons

            # calculate current heating, humidification, and reheat loads
            heating_load = max(0, 1.08*cfm_current['final']*(DA["DB"]-coil_EA["DB"]))
            humid_load = max(0, (60/13.33)*cfm_current['final']*(DA['W']-coil_EA['W']))
            if occupancy == "Y":
                reheat_load = max(0, 1.08*cfm_current['final']*(temp["setpt_occ"]-temp["SAT"]) - cooling_vals["PLR"]*cooling_vals["load_wsf"]*area*3.412)
            else:
                reheat_load = max(0, 1.08*cfm_current['final']*(temp["setpt_unocc"]-temp["SAT"]) - cooling_vals["PLR"]*cooling_vals["load_wsf"]*area*3.412)
            
            # calculate design kW for supply, lab exhaust, general exhaust AHU fans based on component PDs
            fan_kw_design = {'SF':0, 'LEF':0, 'GEF':0}
            for component in PD_vals_current.index:
                for x in ['SF', 'LEF', 'GEF']:
                    fan_kw_design[x] += fan_vals["kw/pd/cfm"]*PD_vals_current[x][component]*cfm_design['final']            

            # calculate current energy use
            energy = {}
            energy['cooling'] = energy_cooling(design_cooling, cfm_design, cooling_load, DA, cooling_vals, OA)
            energy['heating'] = heating_load
            energy['humid'] = humid_load*steam_vals["btu/lb"]
            energy['reheat'] = reheat_load
            
            energy['SF'] = (0.7878787879*airflow_PLR**2 + 0.2181818182*airflow_PLR - 0.0033333333)*fan_kw_design['SF']
            energy['LEF'] = max(0, 0.7878787879*LE_airflow_PLR**2 + 0.2181818182*LE_airflow_PLR - 0.0033333333)*fan_kw_design['LEF']
            energy['GEF'] = max(0, 0.7878787879*GE_airflow_PLR**2 + 0.2181818182*GE_airflow_PLR - 0.0033333333)*fan_kw_design['GEF']
            energy['fans'] = energy['SF'] + energy['LEF'] + energy['GEF']
            
            energy['HW pump'] = energy_HW_pump(design_heating, cfm_design, heating_load, DA)
            energy['CHW pump'] = energy_CHW_pump(design_cooling, cfm_design, cooling_load, cooling_vals, DA)
            energy['CW pump'] = 0
            energy['pumps'] = energy['HW pump'] + energy['CHW pump'] + energy['CW pump']

            # convert kWh to MWh and sum
            totals_energy_elec['cooling'] += energy['cooling']/1000
            totals_energy_elec['fans'] += energy['fans']/1000
            totals_energy_elec['pumps'] += energy['pumps']/1000

            # convert Btu to MMBtu and sum
            totals_energy_gas['heating'] += energy['heating']/1000000
            totals_energy_gas['reheat'] += energy['reheat']/1000000
            totals_energy_gas['humid'] += energy['humid']/1000000

            # calculate current energy use cost and add to running total
            totals_cost['cooling'] += df_elec_utility.at[hr,0]*energy['cooling']
            totals_cost['heating'] += (energy['heating']*utility['$/MMBTU'])/(1000000*heating_vals['efficiency']) # using 1037 BTU/CF // OR convert to MMBTU
            totals_cost['humid'] += (energy['humid']*utility['$/MMBTU'])/(1000000*steam_vals['efficiency'])
            totals_cost['reheat'] += (energy['reheat']*utility['$/MMBTU'])/(1000000*heating_vals['efficiency'])
            totals_cost['fans'] += df_elec_utility.at[hr,0]*energy['fans']
            totals_cost['pumps'] += df_elec_utility.at[hr,0]*energy['pumps']

            # add airflows and current cooling load to running total
            totals_load['cfm_design'] += cfm_design["final"]
            totals_load['cfm_actual'] += cfm_current["final"]
            totals_load['cooling_tons'] += cooling_load["total"]

        # calculate total energy use costs for this case
        totals_energy_cost['electricity'] = totals_cost['cooling'] + totals_cost['fans'] + totals_cost['pumps']
        totals_energy_cost['gas'] = totals_cost['heating'] + totals_cost['humid'] + totals_cost['reheat']
        totals_energy_cost['total'] = totals_energy_cost['electricity'] + totals_energy_cost['gas']

        totals_energy_elec['sum'] = totals_energy_elec['cooling'] + totals_energy_elec['fans'] + totals_energy_elec['pumps']
        totals_energy_gas['sum'] = totals_energy_gas['heating'] + totals_energy_gas['humid'] + totals_energy_gas['reheat']

        # calculate carbon dioxide equivalent emissions
        for a, b in totals_energy_elec.items():
            for c, d in GWP.items():
                total_CO2e[a] += d*elec_emissions[c]*b
        for a, b in totals_energy_gas.items():
            for c, d in GWP.items():
                total_CO2e[a] += d*gas_emissions[c]*b

        # add calculated values to output list
        cases = compile_items(cases, totals_load, totals_energy_cost, totals_cost, driving_factor, total_CO2e)
 
    df_cases = pd.DataFrame(cases).transpose()
    df_cases.columns = ['Fume Hood Count / Size', 'Laboratory Type', 'Fume Hood Usage', 'Fume Hood Type','Cooling Load (W/SF)', 'Minimum Occupied ACH',
                        'Exhaust Energy Recovery', 'DCV (Aircuity)', 'Chilled Beams', 'AHU Design Velocity', 'Point Exhaust Count', 'Point Exhaust Control',
                        'Design Airflow (Total)', 'Actual Airflow (Total)', 'Cooling Hours', 'ACHR Hours', 'Exhaust Hours',
                        'Central Cooling Tons', 'Central Cooling Energy Cost', 'Central Heating Energy Cost',
                        'Central Humidification Energy Cost', 'Reheat Energy Cost', 'Fan Cost', 'Pump Cost',
                        'Total Electricity Energy Cost', 'Total Gas Energy Cost', 'Total Energy Cost', 'Cooling CO2e',
                        'Heating CO2e', 'Humidification CO2e', 'Reheat CO2e', 'Fan CO2e', 'Pump CO2e', 'Total Emissions (kg-CO2e)']
    # provide output path here
    df_cases.to_excel(f"C:\\Users\\uirfan\\OneDrive - SmithGroup Companies Inc\\Desktop\\14276 - UWM eng\\results_0FH(2)_{q[0][1][:q[0][1].find('-')-1]}.xlsx")

In [169]:
def compile_items(cases, totals_load, totals_energy_cost, totals_cost, driving_factor, total_CO2e):
    cases[12].append(totals_load['cfm_design']) 
    cases[13].append(totals_load['cfm_actual'])
    cases[14].append(driving_factor['Cooling'])
    cases[15].append(driving_factor["ACHR"])
    cases[16].append(driving_factor["Exhaust"])
    cases[17].append(totals_load['cooling_tons'])
    cases[18].append(totals_cost['cooling'])
    cases[19].append(totals_cost['heating'])
    cases[20].append(totals_cost['humid'])
    cases[21].append(totals_cost['reheat'])
    cases[22].append(totals_cost['fans'])
    cases[23].append(totals_cost['pumps'])
    cases[24].append(totals_energy_cost['electricity'])
    cases[25].append(totals_energy_cost['gas'])
    cases[26].append(totals_energy_cost['total'])
    cases[27].append(total_CO2e['cooling'])
    cases[28].append(total_CO2e['heating'])
    cases[29].append(total_CO2e['humid'])
    cases[30].append(total_CO2e['reheat'])
    cases[31].append(total_CO2e['fans'])
    cases[32].append(total_CO2e['pumps'])
    cases[33].append(total_CO2e['sum'])
    return cases

In [170]:
## calculate cooling airflow in current hour
def airflow_cooling_hourly(cooling_vals, area, temp, occupancy):
    cooling_load_btu = cooling_vals["load_wsf"]*area*3.412
    cooling_load_btu_current = cooling_load_btu*cooling_vals["PLR"]
    if occupancy == "Y":
        return round_multiple((cooling_load_btu_current)/(1.08*(temp["setpt_occ"]-temp["SAT"])),5)
    else:
        return round_multiple((cooling_load_btu_current)/(1.08*(temp["setpt_unocc"]-temp["SAT"])),5)

In [171]:
## calculate design cooling airflow
def airflow_cooling_design(cooling_load_wsf, area, temp):
    cooling_load_btu = cooling_load_wsf*area*3.412
    return round_multiple((cooling_load_btu)/(1.08*(temp["setpt_occ"]-temp["SAT"])),5)

In [172]:
## calculate ACHR airflow in current hour
def airflow_ACHR_hourly(ACH, height, area, aircuity, occupancy):
    volume = area*height

    if aircuity:
        airflow_occ = ACH["aircuity"]*volume/60
    else:
        airflow_occ = ACH["min_occ"]*volume/60
        
    if occupancy == "Y":
        return airflow_occ
    else:
        return min(ACH["min_unocc"]*volume/60, airflow_occ)

In [173]:
## calculate design ACHR airflow
def airflow_ACHR_design(ACH, height, area):
    volume = area*height
    return ACH["min_occ"]*volume/60

In [174]:
## calculate makeup airflow for fume hood exhaust in current hour
def airflow_LE_hourly(fumehood, point_exhaust, cfm_transfer):
    diversified_airflow = math.ceil(fumehood["cfm_fraction"]*fumehood["cfm_max"]*fumehood['qty'])
    FHE_actual = max(fumehood["cfm_min"]*fumehood['qty'], diversified_airflow)
    PE_actual = point_exhaust['qty']*point_exhaust['cfm']*point_exhaust['fraction']
    
    cfm_makeup = FHE_actual + PE_actual - cfm_transfer
    return cfm_makeup

In [175]:
## calculate design fume hood exhaust airflow
def airflow_LE_design(fumehood, point_exhaust, cfm_transfer):
    diversified_airflow = math.ceil(1*fumehood["cfm_max"]*fumehood['qty'])
    FHE_actual = max(fumehood["cfm_max"]*fumehood['qty'], diversified_airflow)
    PE_actual = point_exhaust['qty']*point_exhaust['cfm']*1

    cfm_makeup = FHE_actual + PE_actual - cfm_transfer
    return cfm_makeup

In [176]:
## ceiling function for rounding to nearest 5
def round_multiple(num, multiple):
    return int(math.ceil(num/multiple)*multiple)

In [177]:
## calculate cooling load (sensible & latent)
def load_cooling(cfm, setpt, DA):
    cooling_load = {}
    for i in [[1.08, "DB", "sensible"], [4760, "W", "latent"]]:
        cooling_load[i[2]] = max(0, i[0]*cfm['final']*(setpt[i[1]]-DA[i[1]]))
    return cooling_load

In [178]:
## calculate hot water pump kW
def energy_HW_pump(design_heating, cfm_design, heating_load, DA):
    heating_load_design = 1.08*cfm_design['final']*(DA["DB"]-design_heating["DB"])
    pump_design_kw = heating_load_design*24/(500*10*1000) # upsized to 10 and 24 -- power = (Q/500*dT)*19, using dT = 20F and 19 W/GPM from ASHRAE 90.1 G3.1.3.5
    heating_load_PLR = heating_load/heating_load_design
    
    # coefficients taken from power vs CFM PLR curve in trane trace (variable vol. CHW pump)
    part_load_factor = max(0, (0.000486*heating_load_PLR)+(0.56138*heating_load_PLR**2)-(1.1412*heating_load_PLR**3)+(2.2214*heating_load_PLR**4)-(0.64552*heating_load_PLR**5))
    return part_load_factor*pump_design_kw

In [179]:
## calculate chilled water pump kW
def energy_CHW_pump(design_cooling, cfm_design, cooling_load, cooling_vals, DA):
    cooling_load_design = load_cooling(cfm_design, design_cooling, DA)
    cooling_load_design['total'] = sum(cooling_load_design.values())
    
    pump_design_kw = cooling_load_design['total']*18/(500*10*1000) # upsized to 10 and 18 -- power = (Q/500*dT)*13, using dT = 20F and 13 W/GPM from ASHRAE 90.1 G3.1.3.10
    cooling_load_PLR = cooling_load['total']/cooling_load_design['total']
    
    # coefficients taken from power vs CFM PLR curve in trane trace (variable vol. CHW pump)
    part_load_factor = max(0, (0.000486*cooling_load_PLR)+(0.56138*cooling_load_PLR**2)-(1.1412*cooling_load_PLR**3)+(2.2214*cooling_load_PLR**4)-(0.64552*cooling_load_PLR**5))
    return part_load_factor*pump_design_kw*cooling_vals["tower/pump_factor"]

In [180]:
## calculate condenser water pump kW
def energy_CW_pump():
    pass

In [181]:
## calculate cooling kW
def energy_cooling(design_cooling, cfm_design, cooling_load, DA, cooling_vals, OA):
    cooling_energy = {}
    cooling_load_design = load_cooling(cfm_design, design_cooling, DA)
    cooling_load_design['total'] = sum(cooling_load_design.values())/12000
    cooling_energy['design'] = cooling_load_design['total']*cooling_vals['efficiency']

    cooling_load_PLR = cooling_load['total']/cooling_load_design['total']
    if cooling_load_PLR > 0:
        # coefficients taken from power vs CFM PLR curve in trane trace
        part_load_factor = 0.6536519037*cooling_load_PLR**3 - 0.5544871795*cooling_load_PLR**2 +0.7992618493*cooling_load_PLR + 0.1024125874
    else:
        part_load_factor = 0
        
    if OA["DB"] > 85:
        ambient_factor = 1
    elif OA["DB"] < 55:
        ambient_factor = 0.9
    else:
        ambient_factor = 0.00333333*OA["DB"]+0.7166666666
        
    cooling_energy['final'] = part_load_factor*ambient_factor*cooling_energy['design']
    # cooling_energy['tower/pump'] = cooling_vals["tower/pump_factor"]*cooling_energy['sum']
    # cooling_energy['final'] = cooling_energy['tower/pump']+cooling_energy['sum']
    return cooling_energy['final']

In [182]:
## calculate leaving air conditions through energy recovery (both runaround & wheel)
def energy_recovery(effectiveness, OA, OA_cfm, EA, EA_cfm):
    OA["m_da"] = OA_cfm/OA["v"] # calculate dry mass flow rate of OA
    EA["m_da"] = EA_cfm/EA["v"] # calculate dry mass flow rate of EA
    SA = {}
    SA["DB"] = OA["DB"] - effectiveness["sensible"]*(min(OA["m_da"],EA["m_da"])/OA["m_da"])*(OA["DB"]-EA["DB"])
    SA["W"] = OA["W"] - effectiveness["latent"]*(min(OA["m_da"],EA["m_da"])/OA["m_da"])*(OA["W"]-EA["W"])
    SA["v"] = 0.370486*(SA["DB"]+459.67)*(1+1.6078*SA["W"])*(1/14.7)
    return SA

In [183]:
## adjust leaving air conditions through energy recovery to prevent overheating
def adjust_recovery(ERC, OA, DA):
    adjusted_ERC = {}
    for i in ["DB", "W"]:
        if OA[i] < DA[i]:
            if ERC[i] < DA[i]:
                adjusted_ERC[i] = ERC[i]
            else:
                adjusted_ERC[i] = DA[i]
        else:
            adjusted_ERC[i] = ERC[i]
    return adjusted_ERC